In [81]:
#Imports
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime


#Constants
url = "http://www.newyorksocialdiary.com/party-pictures?page="
base_url = "http://www.newyorksocialdiary.com"
stop_date = datetime.strptime('Monday, December 1, 2014', '%A, %B %d, %Y')
#base_url_2 = http://www.newyorksocialdiary.com/party-pictures?page=1
match = "pictures"


#Initialize lists
link_list=[]
date_list=[] 
valid_links=[]

#Obtain all the pages and check for valid captions.  First page skipped because of stop date.
for p in range(1,27):  
    response = requests.get(url+str(p))
    c = response.content
    soup = BeautifulSoup(c)

    test = soup.findAll(attrs={'class': 'field-content'})

    for link in test:
        link_string = str(link) 
        if re.search(match, link_string):
            href_string = str(link.next.get("href"))
            link_list.append(href_string)
        else:
            date_list.append(str(link.next))
        

In [82]:

#Get only those captions within the established date range.
for i in range(0,len(date_list)):
    date = datetime.strptime(date_list[i],'%A, %B %d, %Y')
    if date < stop_date:
        valid_links.append(link_list[i])
print len(valid_links)


1192


In [83]:
print len(valid_links)

#Select those tags and attributes which correspond to a photo caption.
raw_captions=[]
caps=[]

for j in range(0,len(valid_links)):
    new_url=base_url + valid_links[j]
    response2 = requests.get(new_url)
    soup = BeautifulSoup(response2.content)
    caps1 = soup.select('.photocaption')
    caps2 = soup.select('font')
    caps=caps1+caps2
    for link in caps:
        if link.text != "":
            raw_captions.append(link.text)
print len(raw_captions)
    
    


1192
113763


In [84]:
#For convenience pickle the data object.
import pickle

with open('CaptionsB','w') as f:
    pickle.dump(raw_captions,f)
    


In [423]:
#For convenience of time, work with the pickled object.
import pickle
captions = pickle.load(open('CaptionsB', 'r'))



In [424]:
#Eliminate those captions with excessively long length.
valid_captions = []
for item in captions:
    if len(item)<250:
        valid_captions.append(item)
print len(valid_captions)


113553


In [425]:
print valid_captions[113200]



Pat
                    Carey, Joan McGivern, and Mario Buatta


In [426]:
#Eliminate some common words.
test_list = valid_captions
print len(test_list)

Title = ['President', 'president','Board Member','Sir','Honoree', 'Prima ballerina' ,'Trustee','Gala ', 'Event ', 'Girl Scout', 'Special Surgery', 'Dr.','Mrs.', 'Mr.', 'Ms.','Baroness', 'Princess', 'Ambassador','Commissioner', 'Police', 'Lung','Against', 'Foundation', 'Uniting','Mayor', 'Curator','Director', 'General', 'consul General','Honorable']
Nationality =['French', 'German', 'Russian', 'British', 'Spanish', "New York", 'France', 'Embassy', 'American']
Middle = ['Legal', 'Justice', 'friend', 'friends', 'guests', 'Jr. ','Co-men ','Co-man ', 'PhD', 'MD', 'CEO', 'Guest','M.D.', 'Jr.', 'Host','Co-Chairs','Co-Chairmen', 'Chairs', 'Chair','Chairman', 'Chairmen', '-Chairmen']
Common = ['Award', 'The', 'Steering Committee','his wife','man of','of the' ,'Actor','Actress', 'Playwright', 'Broadway', ' her husband ' ,'Have', 'Dream','Committee', 'Company','Dance', 'Cocktail', 'School', 'Annual', 'Gala', 'National', 'Fund', 'Kettering','Education', 'Humanitarian', 'Family', 'Museum','Historic','Natural History','Memorial']

for item in test_list:
    if "Photographs by" in item:
        test_list.remove(item)
for i in range(0, len(test_list)):
    for title in Title:
        if title in test_list[i]:
            test_list[i]=test_list[i].replace(title, "")
    for nat in Nationality:
        if nat in test_list[i]:
            test_list[i]=test_list[i].replace(nat, "")
    for mid in Middle:
        if mid in test_list[i]:
            test_list[i]=test_list[i].replace(mid, "") 
    for common in Common:
        if common in test_list[i]:
            test_list[i]=test_list[i].replace(common,"")
            
print len(test_list)
print test_list[30:40]

113553
112320
[u' Carter Bays and Craig Thomas ', u' Melissa Errico, Todd Hollander, and Natalia Bulgari ', u' George Lichtblau, Anne Lichtblau, and  Sig Ackerman ', u' Daniel Murphy, Deann Murphy, Jessica Farrell, and Ken Farrell ', u' Jonny Podell ', u' Ashley McDermott ', u' Caroline Dean ', u' Jena Silverman, John Silverman, Tess Silverman, and Dara Silverman ', u' Jaime Gleicher and Jared Seligman ', u' Fred Feiner, Ricki Feiner, Sandie Greene, and Bob Greene ']


In [427]:
#Split each caption entry on commas.
revised_list=[]
for y in test_list:
    revised_list.append(y.split(","))

print len(revised_list)
print revised_list[30:40]

112320
[[u' Carter Bays and Craig Thomas '], [u' Melissa Errico', u' Todd Hollander', u' and Natalia Bulgari '], [u' George Lichtblau', u' Anne Lichtblau', u' and  Sig Ackerman '], [u' Daniel Murphy', u' Deann Murphy', u' Jessica Farrell', u' and Ken Farrell '], [u' Jonny Podell '], [u' Ashley McDermott '], [u' Caroline Dean '], [u' Jena Silverman', u' John Silverman', u' Tess Silverman', u' and Dara Silverman '], [u' Jaime Gleicher and Jared Seligman '], [u' Fred Feiner', u' Ricki Feiner', u' Sandie Greene', u' and Bob Greene ']]


In [428]:
#Take care of 'with' in caption by splitting into two strings.
for foto_element in revised_list:
    for t in range(0, len(foto_element)):
        if ' with ' in foto_element[t]:
            hold = foto_element[t].split('with')
            foto_element[t]=hold[0]
            foto_element.append(hold[1])


In [429]:
#Take care of 'and' in caption.  Three cases: before name, betweeen names married, and between names.
for foto_caption in revised_list:
    for s in range(0, len(foto_caption)):
        if ' and ' in foto_caption[s]:
            temp = foto_caption[s].split()
            if temp[0] == 'and':
                foto_caption[s] = " ".join(temp[1:])
            else:
                if temp[1] == 'and':
                    foto_caption[s] = " ".join(temp[2:])
                    foto_caption.append(" ".join([temp[0], temp[-1]]))
                else:
                    if temp[2] == 'and':
                        foto_caption[s] = " ".join(temp[3:])
                        foto_caption.append(" ".join(temp[0:2]))
                    else:
                        if len(temp)>4:
                            if temp[3]== 'and':
                                foto_caption[s] = " ".join(temp[4:])
                                foto_caption.append(" ".join(temp[0:3]))
                                                                                           
#Get rid of empty spaces at start of name.               
for clean_caption in revised_list:
    for r in range(0, len(clean_caption)):
        if clean_caption[r] != "" and clean_caption[r][0] == " ":
            clean_caption[r] = clean_caption[r][1:]
            
            
print revised_list[30:40]
print len(revised_list)

[[u'Craig Thomas', u'Carter Bays'], [u'Melissa Errico', u'Todd Hollander', u'Natalia Bulgari'], [u'George Lichtblau', u'Anne Lichtblau', u'Sig Ackerman'], [u'Daniel Murphy', u'Deann Murphy', u'Jessica Farrell', u'Ken Farrell'], [u'Jonny Podell '], [u'Ashley McDermott '], [u'Caroline Dean '], [u'Jena Silverman', u'John Silverman', u'Tess Silverman', u'Dara Silverman'], [u'Jared Seligman', u'Jaime Gleicher'], [u'Fred Feiner', u'Ricki Feiner', u'Sandie Greene', u'Bob Greene']]
112320


In [430]:
#Take care of long sentences by taking the first two names.
for foto_item in revised_list:
    for u in range(0, len(foto_item)):
        special = foto_item[u].split()
        if len(special)>4:
            foto_item[u]=special[0]+ " " + special[1]


In [431]:
#Remove empty lists.
for picture in revised_list:
    if picture == []:
        revised_list.remove(picture)
print len(revised_list)

112320


In [432]:
#Remove odd characters.

for foto_check in revised_list:
    for item in foto_check:
        if item == " ":
            foto_check.remove(item)
        if item == "  ":
            foto_check.remove(item)
        if item == "s":
            foto_check.remove(item)
        if item == 'a ':
            foto_check.remove(item)
            
            #if picture_item[v] == "  ":
                #picture_item.remove(picture_item[v])
            #if picture_item[v] == 's':
                #picture_item.remove(picture_item[v])

for foto_check2 in revised_list:
    for item2 in foto_check2:
        if item2 == '':
            foto_check2.remove(item2)
            
                   
print revised_list[30:40]

[[u'Craig Thomas', u'Carter Bays'], [u'Melissa Errico', u'Todd Hollander', u'Natalia Bulgari'], [u'George Lichtblau', u'Anne Lichtblau', u'Sig Ackerman'], [u'Daniel Murphy', u'Deann Murphy', u'Jessica Farrell', u'Ken Farrell'], [u'Jonny Podell '], [u'Ashley McDermott '], [u'Caroline Dean '], [u'Jena Silverman', u'John Silverman', u'Tess Silverman', u'Dara Silverman'], [u'Jared Seligman', u'Jaime Gleicher'], [u'Fred Feiner', u'Ricki Feiner', u'Sandie Greene', u'Bob Greene']]


In [433]:
#revised_list=[]
#for i in range(0,len(test_list)):
    #r = re.findall(r'([A-Z]{1}[a-z]*\s[A-Z]{1}[a-z]+)', test_list[i])
    #r = re.findall(r'([A-Z]{1}[a-z]*\s[A-Z]{1}[c]*[A-Z]*[a-z]+)', test_list[i])
    #if r != []:
        #revised_list.append(r)
#print len(revised_list)
#print revised_list[30:40]

In [434]:
#Use itertools to create all of the combinations between people in the same foto.
import itertools

combinations_list=[]
for j in range(0,len(revised_list)):
    results = [x for x in itertools.combinations(revised_list[j], 2)]
    if results != []:
        combinations_list+=results
print len(combinations_list)
print combinations_list[170:190]

237354
[(u'Sharon Teles', u'Eby McKay'), (u'Carol Sutton Lewis', u'Kathryn Chenault'), (u'Dan Paduano', u'Nancy Paduano'), (u'George Hambrecht', u'Andrea Fahnestock'), (u'Doug Blonsky', u'Judy Angelo'), (u'James Scott', u'Barbara Scott'), (u'James Scott', u'Julie Miller'), (u'James Scott', u'Kamie Lightburn'), (u'Barbara Scott', u'Julie Miller'), (u'Barbara Scott', u'Kamie Lightburn'), (u'Julie Miller', u'Kamie Lightburn'), (u'Anne Harrison', u'Bill Harrison'), (u'Mary Silver', u'Mitchell Silver'), (u'William Zeckendorf', u'Gillian Miniter'), (u'Tom Gold', u'Michelle Smith'), (u'Patti Harris', u'Tom Kempner'), (u'Patti Harris', u'Veronica White'), (u'Tom Kempner', u'Veronica White'), (u'Marcus Miller', u'Jon Batiste'), (u'Geoffrey Canada', u'Sherrilyn Ifill')]


In [435]:
#Create graph to map all of the connections.
import networkx as nx
G=nx.Graph()
G.add_edges_from(combinations_list)

In [436]:
#Get information about the number of pictures for individuals.
listing_dict = dict(nx.degree(G))
listing = list(nx.degree(G))
print listing[800]
print len(listing_dict)
print type(listing_dict)
print listing_dict['Douglas Jencks']


Robert Cecot
106814
<type 'dict'>
3


In [437]:
#Create a dataframe of the information.
import pandas as pd
import numpy as np

#hope = pd.DataFrame.from_dict(listing_dict)
df = pd.DataFrame([[key,value] for key,value in listing_dict.iteritems()],columns=["Name","Degrees"])


print df.describe()

             Degrees
count  106814.000000
mean        3.828131
std         6.746784
min         1.000000
25%         1.000000
50%         2.000000
75%         4.000000
max       425.000000


In [438]:
#print df

#Sort the list for top 100 people with connections.
top_df = df.sort(['Degrees'], ascending=[False])[:100]
top_df = top_df.reset_index(drop=True)

print top_df[:100]

print top_df.describe()

                     Name  Degrees
0          Jean Shafiroff      425
1         Mark Gilbertson      348
2         Gillian Miniter      244
3      Geoffrey Bradfield      209
4     Alexandra Lebenthal      209
5            Mario Buatta      204
6           Somers Farkas      202
7           Andrew Saffir      201
8               Alina Cho      190
9             Sharon Bush      189
10          Yaz Hernandez      187
11        Kamie Lightburn      181
12        Debbie Bancroft      178
13     Lucia Hwong Gordon      174
14       Eleanora Kennedy      173
15       Patrick McMullan      165
16    Muffie Potter Aston      162
17          Allison Aston      159
18         Bettina Zilkha      158
19       Amy Fine Collins      147
20         Martha Stewart      147
21       Deborah Norville      143
22          Barbara Tober      142
23        Kipton Cronkite      141
24                             138
25          Grace Meigher      138
26            Lydia Fenet      136
27          Jamee Gr

In [439]:
#Initialize the answer tuple for question one.
ANSWER1 = []
#Get the required values for the answer tuple.
for i in top_df.index:
    tup1=(top_df.iloc[i,0], top_df.iloc[i,1])
    ANSWER1.append(tup1)
print ANSWER1


[(u'Jean Shafiroff', 425), (u'Mark Gilbertson', 348), (u'Gillian Miniter', 244), (u'Geoffrey Bradfield', 209), (u'Alexandra Lebenthal', 209), (u'Mario Buatta', 204), (u'Somers Farkas', 202), (u'Andrew Saffir', 201), (u'Alina Cho', 190), (u'Sharon Bush', 189), (u'Yaz Hernandez', 187), (u'Kamie Lightburn', 181), (u'Debbie Bancroft', 178), (u'Lucia Hwong Gordon', 174), (u'Eleanora Kennedy', 173), (u'Patrick McMullan', 165), (u'Muffie Potter Aston', 162), (u'Allison Aston', 159), (u'Bettina Zilkha', 158), (u'Amy Fine Collins', 147), (u'Martha Stewart', 147), (u'Deborah Norville', 143), (u'Barbara Tober', 142), (u'Kipton Cronkite', 141), (u'', 138), (u'Grace Meigher', 138), (u'Lydia Fenet', 136), (u'Jamee Gregory', 135), (u'Christopher Hyland', 132), (u'Elizabeth Stribling', 131), (u'Karen LeFrak', 129), (u'Donna Karan', 128), (u'Evelyn Lauder', 127), (u'Russell Simmons', 127), (u'Diana Taylor', 127), (u'Paula Zahn', 126), (u'Nicole Miller', 126), (u'Alec Baldwin', 124), (u'Leonard Lauder',

In [440]:
#Obtain the pagerank information.
pr_dict = nx.pagerank_scipy(G, alpha=0.85)

pr_df = pd.DataFrame([[key,value] for key,value in pr_dict.iteritems()],columns=["Name","Rank"])


In [441]:
#Sort the list according to top ranks.
top_pr_df = pr_df.sort(['Rank'], ascending=[False])[:100]
top_pr_df = top_pr_df.reset_index(drop=True)

print top_pr_df[76:100]

print top_df.describe()

                        Name      Rank
76               Jamie Niven  0.000127
77        Beth Rudin DeWoody  0.000126
78      Melissa Berkelhammer  0.000126
79            Clockwise from  0.000126
80             Amy McFarland  0.000126
81               Denise Rich  0.000124
82      Cassandra Seidenfeld  0.000124
83              R. Couri Hay  0.000124
84                Tina Brown  0.000124
85             Mary Davidson  0.000123
86                Agnes Gund  0.000123
87           Daniel Benedict  0.000122
88            Wendy Carduner  0.000121
89            Cynthia Lufkin  0.000120
90          Margaret Russell  0.000120
91             Mary Van Pelt  0.000119
92          Bettina Prentice  0.000119
93         Chuck Scarborough  0.000118
94          Edward Callaghan  0.000118
95        Kristian Laliberte  0.000118
96                CeCe Black  0.000117
97  Dayssi Olarte de Kanavos  0.000116
98         Michael Bloomberg  0.000115
99               Peter Lyden  0.000115
          Degrees
count  

In [442]:
#Initialize the tuple for the second question.
ANSWER2 = []
#Get the required values for the answer tuple.
for i in top_pr_df.index:
    tup2=(top_pr_df.iloc[i,0], top_pr_df.iloc[i,1])
    ANSWER2.append(tup2)
print ANSWER2

[(u'Jean Shafiroff', 0.00051017717120291437), (u'Mark Gilbertson', 0.00042461662649169788), (u'Gillian Miniter', 0.00029455200988996505), (u'Mario Buatta', 0.00028239832773155967), (u'Andrew Saffir', 0.00028018032708199342), (u'Geoffrey Bradfield', 0.00026674072552197519), (u'Alexandra Lebenthal', 0.00025344656741694307), (u'Sharon Bush', 0.00024982846872476073), (u'Somers Farkas', 0.0002394643294944174), (u'Yaz Hernandez', 0.00022583236687775897), (u'Alina Cho', 0.00022452905819222069), (u'Lucia Hwong Gordon', 0.00022072829353856466), (u'Kamie Lightburn', 0.00021944586164993076), (u'Patrick McMullan', 0.00021410288179690465), (u'Debbie Bancroft', 0.00021216768767288449), (u'Barbara Tober', 0.00020779579260971136), (u'Eleanora Kennedy', 0.00020143041459213086), (u'Christopher Hyland', 0.0001919032665927794), (u'Martha Stewart', 0.000190563856013161), (u'Donna Karan', 0.00018421022139295063), (u'Kipton Cronkite', 0.00018408535325863541), (u'Bettina Zilkha', 0.0001819178213055089), (u'Am

In [443]:
#Create a multigraph for the connections.
import networkx as nx

M=nx.MultiGraph()
M.add_edges_from(combinations_list)

In [444]:
#Create a dictionary with all the connection pairs
friend_dict={}
for pair in combinations_list:
    edge_wt = M.number_of_edges(*pair)
    friend_dict[pair]=edge_wt


In [445]:
#Create a dataframe with the people pairs.
friend_df = pd.DataFrame([[key,value] for key,value in friend_dict.iteritems()],columns=["Pair","Num"])
print friend_df.head()

                            Pair  Num
0     (Lisa Genova, Bill Wright)    1
1  (Anne Ogilvy, Annalise Stack)    2
2   (Hoda Kotb, Jennifer Wexler)    5
3   (Gerold Klauer, Jana Klauer)    4
4      (Mario Batali, Susi Cahn)    2


In [446]:
#Get the top 200 because of the mirror connections of the pairs.
top_friends_df = friend_df.sort(['Num'], ascending=[False])[:200]
top_friends_df = top_friends_df.reset_index(drop=True)
print top_friends_df.describe()

              Num
count  200.000000
mean    27.290000
std     17.009778
min     15.000000
25%     17.000000
50%     21.000000
75%     31.250000
max    122.000000


In [447]:
#Initialize the tuple for question  three.  
ANSWER3 = []
check = 0
#Get the required values for the answer tuple. Eliminate mirror pairs
for i in top_friends_df.index:
    tup3=(top_friends_df.iloc[i,0], top_friends_df.iloc[i,1])
    pair = top_friends_df.iloc[i,0]
    rev_pair = (pair[1],pair[0])
    tup3_rev=(rev_pair, top_friends_df.iloc[i,1])
    if tup3_rev not in ANSWER3 and check<100:
        ANSWER3.append(tup3)
        check+=1
        
print len(ANSWER3)
print ANSWER3


100
[((u'Sylvester Miniter', u'Gillian Miniter'), 122), ((u'Stewart Lane', u'Bonnie Comley'), 82), ((u'Jamee Gregory', u'Peter Gregory'), 73), ((u'Daniel Benedict', u'Andrew Saffir'), 69), ((u'Geoffrey Bradfield', u'Roric Tobin'), 68), ((u'Donald Tober', u'Barbara Tober'), 59), ((u'Jonathan Farkas', u'Somers Farkas'), 56), ((u'Jean Shafiroff', u'Martin Shafiroff'), 52), ((u'Alexandra Lebenthal', u'Jay Diamond'), 52), ((u'Tatiana Platt', u'Campion Platt'), 49), ((u'Michael Kennedy', u'Eleanora Kennedy'), 44), ((u'Sessa von Richthofen', u'Richard Johnson'), 43), ((u'Melissa Morris', u'Chappy Morris'), 43), ((u'Yaz Hernandez', u'Valentin Hernandez'), 43), ((u'Chris Meigher', u'Grace Meigher'), 42), ((u'Barbara Regna', u'Peter Regna'), 42), ((u'Jonathan Tisch', u'Lizzie Tisch'), 41), ((u'Guy Robinson', u'Elizabeth Stribling'), 40), ((u'Margo Catsimatidis', u'John Catsimatidis'), 38), ((u'Karl Wellner', u'Deborah Norville'), 37), ((u'Wilbur Ross', u'Hilary Geary Ross'), 37), ((u'Douglas Han